# Classification based on Quickdraw Dataset

### Imports

In [2]:
import os
from PIL import Image
import json

import random
import numpy as np
import cairocffi as cairo
from sklearn.model_selection import train_test_split

import typing_extensions
from importlib import reload
reload(typing_extensions)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

NameError: name '_C' is not defined

### Functions for the Dataset

In [26]:
# Get the classes that will be used
def load_classes(file_path):
    res = {}
    count = 0
    for line in open(file_path, 'r'):
        res[line.rstrip()] = count
        count+=1
    return res

In [4]:
### The code is taken from the original GitHub of the QuickDrawDataset
def vector_to_raster(vector_images, side=64, line_diameter=16, padding=16, bg_color=(0,0,0), fg_color=(1,1,1)):
    
    original_side = 256.
    
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, side, side)
    ctx = cairo.Context(surface)
    ctx.set_antialias(cairo.ANTIALIAS_BEST)
    ctx.set_line_cap(cairo.LINE_CAP_ROUND)
    ctx.set_line_join(cairo.LINE_JOIN_ROUND)
    ctx.set_line_width(line_diameter)

    # scale to match the new size
    # add padding at the edges for the line_diameter
    # and add additional padding to account for antialiasing
    total_padding = padding * 2. + line_diameter
    new_scale = float(side) / float(original_side + total_padding)
    ctx.scale(new_scale, new_scale)
    ctx.translate(total_padding / 2., total_padding / 2.)

    raster_images = []
    for vector_image in vector_images:
        # clear background
        ctx.set_source_rgb(*bg_color)
        ctx.paint()
        
        bbox = np.hstack(vector_image).max(axis=1)
        offset = ((original_side, original_side) - bbox) / 2.
        offset = offset.reshape(-1,1)
        centered = [stroke + offset for stroke in vector_image]

        # draw strokes, this is the most cpu-intensive part
        ctx.set_source_rgb(*fg_color)        
        for xv, yv in centered:
            ctx.move_to(xv[0], yv[0])
            for x, y in zip(xv, yv):
                ctx.line_to(x, y)
            ctx.stroke()

        data = surface.get_data()
        raster_image = np.copy(np.asarray(data)[::4])
        raster_images.append(raster_image)
    
    return raster_images

In [14]:
# return the data as an array containing images as 1D arrays
def load_data(sample, n_images, dimension):
    sample_data = [json.loads(line) for line in open(f'data/full_simplified_{sample}.ndjson', 'r')]
    sample_data = random.sample(sample_data, k=n_images)
    vector_images = [drawing_data['drawing'] for drawing_data in sample_data]
    return np.array(vector_to_raster(vector_images, side=dimension))

In [6]:
# Saves the datasets as images in the "images" folder
def save_png(drawing_class, data, dimension):
    if not os.path.exists(f"images/{drawing_class}"): 
        os.makedirs(f"images/{drawing_class}")
    
    count = 0
    for image_arr in data:
        image_arr = np.reshape(image_arr, (dimension, -1))
        img = Image.fromarray(image_arr, "L")
        img.save(f"images/{drawing_class}/axe_{count}.png")
        count += 1

### Functions for the network

### Main part

Variables describing the data

In [34]:
# Images will be 64x64
img_dim = 64
# Number of images taken for each animal
n_images = 2
# Proportion used for to train the model
train_prop = 0.8

Store the classes in a dictionnary with their index

In [29]:
classes = load_classes("class_names.txt")
n_class = len(classes)
print(n_class)
print(classes)

2
{'axe': 0, 'bicycle': 1}


Build the different sets from the data

In [35]:
data_X = []
data_Y = []
for drawing_class in classes:
    data = load_data(drawing_class, n_images, dimension = img_dim)
    data_X.append(data)
    data_Y.append(np.full(n_images, classes[drawing_class]))





# maybe not necesssary
train_images, test_images, train_labels, test_labels = train_test_split(data_X, data_Y, train_size = train_prop, random_state = 42, shuffle = True)



In [ ]:
preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std_dev)
        ])

In [ ]:
train_dataset = Dataset(train_data, transform=preprocess)
test_dataset = Dataset(test_data, transform=preprocess)

train_loader = torch.utils.data.DataLoader(dataset1, batch_size = , )

In [ ]:
train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

For the datas :
- load in arrays
- sort to keep only the recognized drawings
- (save the arrays)
- (possibility to save in png)
- faire des sets de tests et train

For the CNN:
- (batch normailzation)
- conv relu pooling conv relu pooling dropout

For vizualization and documentation:
- Confusion Matrix
- evolution of the error with epochs


### The model